In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from tkinter import Tk, Button, Label, filedialog, messagebox, Canvas, StringVar, OptionMenu  # Add StringVar and OptionMenu from tkinter
from PIL import Image, ImageTk  # Import Image and ImageTk from PIL
from plyer import notification
import os
import threading
import winsound
import shutil
import time
import datetime

# Load the trained emotion detection model for file recognition
emotion_model = load_model('C:/Users/amc/emotion_model3.h5')

# Map emotion labels to human-readable names
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Initialize Tkinter
root = Tk()
root.title("Emotion Detection")

# Global variables
recognition_active = False
notification_flag = True
last_notification_time = 0
cooldown_duration = 5  # Cooldown duration for notifications (in seconds)

# Beep sound configuration
beep_frequency = 2500  # Frequency of the beep sound in Hz
beep_duration = 100  # Duration of the beep sound in milliseconds

# Initialize the SSD model for face detection
ssd_net = cv2.dnn.readNetFromCaffe('deploy.prototxt', 'res10_300x300_ssd_iter_140000_fp16.caffemodel')

# Thresholds for different recognition modes
threshold_file_recognition = 0.2
threshold_realtime_recognition = 0.3
threshold_video_recognition = 0.3

# Add background image
background_image = Image.open('C:/Users/amc/BG.jpg')  # Change 'background_image.jpg' to your image file
background_photo = ImageTk.PhotoImage(background_image)
background_label = Label(root, image=background_photo)
background_label.place(relwidth=1, relheight=1)  # Place the background label to cover the entire window

# Load and resize the banner image
banner_image = Image.open('C:/Users/amc/B3.jpg')  # Change 'path_to_banner_image.jpg' to your actual image path
banner_image = banner_image.resize((600, 100))  # Adjust the size as needed

# Convert the banner image to a PhotoImage object
banner_photo = ImageTk.PhotoImage(banner_image)

# Create a Label widget to display the banner image
banner_label = Label(root, image=banner_photo)
banner_label.pack()


# Function to recognize emotion from an image using MTCNN for file recognition
def recognize_emotion_file(img, file_path):
    detector = MTCNN()
    faces = detector.detect_faces(img)

    # Check if faces are detected
    if len(faces) == 0:
        show_notification("No Faces Detected", "No faces were found in the selected image.")
        return

    for result in faces:
        x, y, w, h = result['box']
        x, y = max(0, x), max(0, y)
        face_roi = img[y:y + h, x:x + w]
        gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        resized_face = cv2.resize(gray_face, (48, 48))
        normalized_face = resized_face.astype('float32') / 255.0
        normalized_face = np.expand_dims(normalized_face, axis=-1)
        emotion_probabilities = emotion_model.predict(np.expand_dims(normalized_face, axis=0))[0]
        predicted_emotion_index = np.argmax(emotion_probabilities)
        confidence = np.max(emotion_probabilities)

        # Check if confidence exceeds threshold
        if confidence >= threshold_file_recognition:
            predicted_emotion = emotion_labels[predicted_emotion_index]
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            label = f"{predicted_emotion.capitalize()} ({confidence:.2f})"
            cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # Check for desired emotion
            if predicted_emotion == "Angry" and notification_flag and is_notification_cooldown_passed():
                show_notification("Angry Emotion Detected", f"Person is {predicted_emotion}!")
                play_beep_sound()
                set_notification_cooldown()

    # Display the annotated image with the option to save
    display_image_with_save_option(img, file_path)


# Function to display the annotated image with the option to save
def display_image_with_save_option(img, file_path):
    cv2.imshow('Emotion Detection from Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Ask the user if they want to save the result
    result = messagebox.askyesno("Save Result", "Do you want to save the annotated image?")
    if result:
        # Save the annotated image with detected emotions
        save_path = save_results_folder(file_path, "Picture results")
        cv2.imwrite(save_path, img)
        show_notification("Result Saved", f"Annotated image saved as {save_path}")


# Function to perform emotion recognition from a selected file
def recognize_from_file():
    file_path = filedialog.askopenfilename(title="Select Image File", filetypes=[("Image files", ".jpg;.png")])
    if file_path:
        image = cv2.imread(file_path)
        if image is not None:
            recognize_emotion_file(image, file_path)
        else:
            messagebox.showerror("Error", "Failed to open the selected file.")
    else:
        messagebox.showinfo("Info", "No file selected.")


# Function to save results folder
def save_results_folder(file_path, folder_name):
    # Desktop path
    desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')

    # Picture results folder path
    results_folder_path = os.path.join(desktop_path, folder_name)

    # Create the folder if it doesn't exist
    if not os.path.exists(results_folder_path):
        os.makedirs(results_folder_path)

    # Generate timestamp for the file name
    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # Save the annotated image with detected emotions
    save_path = os.path.join(results_folder_path, f"{timestamp}_annotated_{os.path.basename(file_path)}")
    return save_path


# Function to validate file type
def validate_file_type(file_path, allowed_extensions):
    _, file_extension = os.path.splitext(file_path)
    return file_extension.lower() in allowed_extensions


# Function to start real-time recognition
def start_realtime_recognition():
    global recognition_active
    recognition_active = True
    threading.Thread(target=update_frame_realtime).start()


# Function to update the frame for real-time recognition using SSD
def update_frame_realtime():
    cap = cv2.VideoCapture(0)
    frame_number = 0  # Counter to keep track of frames

    # Create directory for real-time results on desktop
    desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
    realtime_results_path = os.path.join(desktop_path, 'real-time_results')
    if not os.path.exists(realtime_results_path):
        os.makedirs(realtime_results_path)

    while recognition_active:
        ret, frame = cap.read()

        if not ret:
            break

        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        ssd_net.setInput(blob)
        detections = ssd_net.forward()

        detected_faces = []

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > threshold_realtime_recognition:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype(int)
                face_roi = frame[startY:endY, startX:endX]

                # Convert face ROI to grayscale
                gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)

                # Resize face ROI to match model input size
                resized_face = cv2.resize(gray_face, (48, 48))

                # Normalize the resized face ROI
                normalized_face = resized_face.astype('float32') / 255.0

                # Expand dimensions to match model input shape
                normalized_face = np.expand_dims(normalized_face, axis=-1)
                normalized_face = np.expand_dims(normalized_face, axis=0)

                detected_faces.append((normalized_face, (startX, startY, endX, endY)))

        if detected_faces:
            emotion_predictions = emotion_model.predict(np.vstack([face[0] for face in detected_faces]))
            for i, (normalized_face, (startX, startY, endX, endY)) in enumerate(detected_faces):
                predicted_emotion_index = np.argmax(emotion_predictions[i])
                confidence = emotion_predictions[i][predicted_emotion_index]
                if confidence >= threshold_realtime_recognition:
                    predicted_emotion = emotion_labels[predicted_emotion_index]
                    label = f"{predicted_emotion.capitalize()} ({confidence:.2f})"
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                    # Check for desired emotion
                    selected_emotion_value = selected_emotion.get()
                    if predicted_emotion == selected_emotion_value:
                        # Save frame where desired emotion is detected
                        frame_path = os.path.join(realtime_results_path, f"frame_{frame_number}.jpg")
                        cv2.imwrite(frame_path, frame)
                        frame_number += 1

                        if notification_flag and is_notification_cooldown_passed():
                            show_notification(f"{selected_emotion_value} Emotion Detected", f"Person is {selected_emotion_value}!")
                            play_beep_sound()
                            set_notification_cooldown()

        cv2.imshow('Real-time Emotion Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# Function to start video recognition
def start_video_recognition():
    global recognition_active
    recognition_active = True
    threading.Thread(target=recognize_from_video).start()


# Function to recognize from video using SSD
def recognize_from_video():
    file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", ".mp4;.avi")])
    if not file_path:
        return

    cap = cv2.VideoCapture(file_path)
    save_path = os.path.join(os.path.expanduser('~'), 'Desktop', 'Video_Results')
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    frame_number = 0

    while recognition_active:
        ret, frame = cap.read()

        if not ret:
            break

        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        ssd_net.setInput(blob)
        detections = ssd_net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > threshold_video_recognition:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype(int)
                face_roi = frame[startY:endY, startX:endX]

                # Convert face ROI to grayscale
                gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)

                # Resize face ROI to match model input size
                resized_face = cv2.resize(gray_face, (48, 48))

                # Normalize the resized face ROI
                normalized_face = resized_face.astype('float32') / 255.0

                # Expand dimensions to match model input shape
                normalized_face = np.expand_dims(normalized_face, axis=-1)
                normalized_face = np.expand_dims(normalized_face, axis=0)

                emotion_probabilities = emotion_model.predict(normalized_face)[0]
                predicted_emotion_index = np.argmax(emotion_probabilities)
                confidence = np.max(emotion_probabilities)

                # Check if confidence exceeds threshold
                if confidence >= threshold_video_recognition:
                    predicted_emotion = emotion_labels[predicted_emotion_index]
                    label = f"{predicted_emotion.capitalize()} ({confidence:.2f})"
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                    # Check for desired emotion
                    selected_emotion_value = selected_emotion.get()
                    if predicted_emotion == selected_emotion_value:
                        # Save annotated frame
                        annotated_frame_path = os.path.join(save_path, f"frame_{frame_number}.jpg")
                        cv2.imwrite(annotated_frame_path, frame)
                        frame_number += 1

                        if notification_flag and is_notification_cooldown_passed():
                            show_notification(f"{selected_emotion_value} Emotion Detected", f"Person is {selected_emotion_value}!")
                            play_beep_sound()
                            set_notification_cooldown()

        cv2.imshow('Emotion Detection from Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    show_notification("Video Recognition Completed", f"Annotated frames saved in {save_path}")


# Function to detect faces using Haar cascades with OpenCV
def detect_faces_opencv(gray):
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces


# Function to stop recognition
def stop_recognition():
    global recognition_active
    recognition_active = False
    show_notification("Recognition Stopped", "Recognition has been stopped.")


# Function to show a notification
def show_notification(title, message):
    notification_title = title
    notification_message = message
    notification.notify(title=notification_title, message=notification_message, app_name="Emotion Detection")


# Function to play a beep sound
def play_beep_sound():
    winsound.Beep(beep_frequency, beep_duration)


# Function to set cooldown for notifications
def set_notification_cooldown():
    global last_notification_time
    last_notification_time = time.time()


# Function to check if notification cooldown has passed
def is_notification_cooldown_passed():
    global last_notification_time
    current_time = time.time()
    return current_time - last_notification_time >= cooldown_duration


# Function to close the Tkinter window
def close_window():
    root.destroy()


# Styling the interface
root.configure(bg='#263238')
root.geometry("800x800")


# Function to style buttons with curved edges
def style_button(button):
    button.config(relief='raised', bd=3, font=('Helvetica', 12, 'bold'), width=20)


# Buttons for file recognition, real-time recognition, video recognition, stop recognition, and close window
file_button = Button(root, text="Recognize from File", command=recognize_from_file, bg='#4CAF50', fg='white')
style_button(file_button)
file_button.pack(pady=10)

size_message = Label(root, text="File size should be less than 1MB", bg='#F5F5F5', font=('Helvetica', 10))
size_message.pack()

realtime_button = Button(root, text="Real-time Recognition", command=start_realtime_recognition, bg='#2196F3', fg='white')
style_button(realtime_button)
realtime_button.pack(pady=10)

video_button = Button(root, text="Recognize from Video", command=start_video_recognition, bg='#FF9800', fg='white')
style_button(video_button)
video_button.pack(pady=10)

stop_button = Button(root, text="Stop Recognition", command=stop_recognition, bg='#F44336', fg='white')
style_button(stop_button)
stop_button.pack(pady=10)

close_button = Button(root, text="Close", command=close_window, bg='#607D8B', fg='white')
style_button(close_button)
close_button.pack(pady=10)

# Drop-down menu for selecting desired emotion
selected_emotion = StringVar(root)
selected_emotion.set("Angry")  # Set default emotion to Angry
emotion_label = Label(root, text="Select Desired Emotion:", bg='#607D8B', fg='white', font=('Helvetica', 12))
emotion_label.pack(pady=20, padx=50)
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotion_menu = OptionMenu(root, selected_emotion, *emotions)
emotion_menu.config(bg='#F44336', font=('Helvetica', 12), width=20)
emotion_menu.pack(pady=20, padx=50)

# Start the Tkinter main loop
root.mainloop()
